## Use feature-weighted rf model on the crcns vim-1 dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
from time import time
from glob import glob
from scipy.io import loadmat
from PIL import Image
from scipy.stats import pearsonr
from hrf_fitting.src.feature_weighted_rf_models import make_rf_table,receptive_fields, model_space, prediction_menu, bigmult
from hrf_fitting.src.feature_weighted_rf_models import train_fwrf_model
from hrf_fitting.src.gabor_feature_dictionaries import gabor_feature_maps

%load_ext autoreload
%autoreload 2
%matplotlib inline

Using gpu device 0: GeForce GTX TITAN X


### Step 0: Load crcns stimuli

#### load crcns stimuli 

In [ ]:
##known stimulus parameters
Ttrn = 1750
Tval = 120
S = 500
T = Ttrn+Tval
train_stim_files = glob('/media/tnaselar/Data/crcns_datasets/vim-1/Stimuli_Trn_FullRes*.mat')
val_stim_file = '/media/tnaselar/Data/crcns_datasets/vim-1/Stimuli_Val_FullRes.mat'
n_image_channels = 1 ##could be 3 for color images.

In [ ]:
##allocate memory for stim
training_stim = np.zeros((Ttrn,S,S),dtype='float32')

##load training stim
cnt = 0
for sl in train_stim_files:
    this_h5 = h5py.File(sl,'r')
    this_train_stim = this_h5['stimTrn']
    this_num_stim = this_train_stim.shape[-1]
    training_stim[cnt:cnt+this_num_stim,:,:] = np.transpose(this_train_stim[:],[2,1,0])
    cnt += this_num_stim
    this_h5.close()
    
##load validation stim
val_h5 = h5py.File(val_stim_file,'r')
validation_stim = np.transpose(val_h5['stimVal'][:],[2,1,0])
val_h5.close()

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.imshow(validation_stim[0,:,:],cmap='gray')
plt.subplot(1,2,2)
plt.imshow(training_stim[0,:,:],cmap = 'gray')

### Step 1: construct feature maps

In [ ]:
n_orientations = 8
deg_per_stimulus = 20
lowest_sp_freq = .2 ##cyc/deg
highest_sp_freq = 1.5
num_sp_freq = 8
pix_per_cycle = 2.13333333
complex_cell = True

print 'D = total number of features = %d' %(n_orientations * num_sp_freq)

#### construct gabor wavelet stack

In [ ]:
gfm = gabor_feature_maps(n_orientations,
                         deg_per_stimulus,
                         (lowest_sp_freq,highest_sp_freq,num_sp_freq),
                         pix_per_cycle=pix_per_cycle,complex_cell=complex_cell)

In [ ]:
gfm.gbr_table.head(num_sp_freq)

In [ ]:
gfm.filter_stack.shape

#### see one of the gabors

In [ ]:
o = 1 ##choose an orientation
plt.imshow(np.real(gfm.filter_stack[o,0,:,:]),cmap='gray')

### Step 2: receptive fields

In [ ]:
deg_per_radius = (0.5, 5, 6) ##rf sizes in degrees (smallest, largest, number of sizes)
spacing = 1. ##spacing between rf's in degrees
rf = receptive_fields(deg_per_stimulus,deg_per_radius,spacing)

In [ ]:
rf.rf_table.head()

In [ ]:
print 'G = number of rf models = %d' %(rf.rf_table.shape[0])

### Step 3: Model space

#### instantiate model space object

In [ ]:
##construct a sample feature dict to initiate model space
##note that gfm expects stimuli to be 4D T x n_color_channels x S x S tensors,
##so if we want to pass just one image we need to add two singleton axes
init_feat_dict = gfm.create_feature_maps(training_stim[0,np.newaxis,np.newaxis,:,:])

In [ ]:
##construct the model space, add a bias feature (all 1's)
ms = model_space(init_feat_dict, rf, add_bias=True)

#### construct training/validation model space tensors

In [ ]:
##loop over training stimuli because feature maps for all training stim. > 48Gb
training_mst = np.zeros((ms.receptive_fields.G, Ttrn, ms.D)).astype('float32')

num_chunks = 2
stim_dx = np.linspace(0,T-1,num=num_chunks+1, endpoint=True,dtype='int')

cnt = 0
for t in stim_dx[1:]:
    this_training_feature_dict = gfm.create_feature_maps(training_stim[cnt:cnt+t,np.newaxis,:,:])
    training_mst[:,cnt:cnt+t,:] = ms.construct_model_space_tensor(this_training_feature_dict,normalize=False)
    cnt += t

##clear up memory
this_training_feature_dict = []

##normalize and save normalization constants
training_mst = ms.normalize_model_space_tensor(training_mst, save=True)


##should work in one shot for because not too big
val_feature_dict = gfm.create_feature_maps(validation_stim[:,np.newaxis,:,:])
validation_mst = ms.construct_model_space_tensor(val_feature_dict)

    

### Step 4: load and package crcns voxel data

In [ ]:
voxel_file = '/media/tnaselar/Data/crcns_datasets/vim-1/EstimatedResponses.mat'
crcns_voxel_data = h5py.File(voxel_file,'r')
crcns_voxel_data.keys()

#### remove nans, becuase this data-set has some. otherwise even one nan will infect gradient for every voxel.

In [ ]:
voxel_data = np.concatenate((crcns_voxel_data['dataValS1'],crcns_voxel_data['dataTrnS1']),axis=0).astype('float32')
no_nan = np.isnan(voxel_data).sum(axis=0) == 0 ##<<only pulled voxels with nans in training data, should pull if nans in val data too.
voxel_data = voxel_data[:,no_nan]
print voxel_data.shape
V = voxel_data.shape[1]

In [ ]:
crcns_voxel_data.close()

#### get training/validation views on voxel_data

In [ ]:
nvox=1000
trnIdx = np.arange(Tval,T)
valIdx = np.arange(0,Tval)
trn_voxel_data = voxel_data[trnIdx,0:nvox]


### Step 5: run that shit.

#### initialize the feature weights

In [ ]:
initial_feature_weights = np.zeros((ms.receptive_fields.G,ms.D,nvox),dtype='float32')

#### train the model!

In [ ]:
fvl,ffw,frf,beh = train_fwrf_model(training_mst,
                 trn_voxel_data,
                 initial_feature_weights,
                 voxel_binsize = nvox,
                 rf_grid_binsize=100,
                 learning_rate=10**(-4.9),
                 max_iters = 100,
                 early_stop_fraction=0.2,
                 report_every = 25)

#### loss histories, all voxels

In [ ]:
_=plt.plot(beh-beh[0,:])


#### view loss history for a few voxels

In [ ]:
_=plt.plot(beh[:,slice(0,-1,20)]-beh[0,slice(0,-1,20)])

In [ ]:
##loss in "final_validation_loss" = last point of "best_error_history"
print np.min(beh[:,-2])
print fvl[-2]

#### diff between first and last point of loss history, all voxels

In [ ]:
_=plt.hist(beh[0,:]-np.min(beh,axis=0),100)
plt.yscale('log')

### Step 6: model analysis and validation

#### histogram of rf models selected for each voxel
models with large indices are small rfs. so looks like we need higher resolution.

In [ ]:
_=plt.hist(frf,ms.receptive_fields.G)
plt.xlabel('bigger-->smaller')

#### sum of all selected rfs. 

In [ ]:
plt.imshow(np.sum(ms.receptive_fields.make_rf_stack(64, min_pix_per_radius=1)[frf,:,:], axis=0), cmap='hot')

#### prediction accuracy for all voxels

In [ ]:
##grab validation data
val_voxel_data = voxel_data[valIdx,0:nvox]

In [ ]:
##generate predictions one voxel at a time
pred = prediction_menu(validation_mst, ffw[np.newaxis,:,:], rf_indices = frf)

In [ ]:
pred.shape

In [ ]:
##get correlation = prediction accuracy
val_cc = []  
for v in range(nvox-1): 
    cc = pearsonr(val_voxel_data[:,v],pred[:,v])
    if not np.isnan(cc[0]):
        val_cc.append(cc[0])

In [ ]:
##histogram of prediction accuracy, all voxels
_=plt.hist(val_cc,100)
plt.yscale('log')